In [ ]:
# (c) Westerveld 2023
# Watermark remover training and development using pytorch library with python
# A convolutional variational autoencoder that takes in an image with a watermark and (ideally) outputs the same image but without a watermark
# The architecture compresses the image then upsamples it, removing the watermark in the process and outputting an image.
# uses CLWD dataset

In [ ]:
# downloads zipped dataset and extracts, takes about 4 mins
# commented out because this is not needed unless testing the dataste specifically

# !gdown 17y1gkUhIV6rZJg1gMG-gzVMnH27fm4Ij

# !pip install pyunpack
# !pip install patool
# from pyunpack import Archive
# Archive("CLWD.rar").extractall("")

Downloading...
From: https://drive.google.com/uc?id=17y1gkUhIV6rZJg1gMG-gzVMnH27fm4Ij
To: /content/CLWD.rar
100% 3.35G/3.35G [00:52<00:00, 63.3MB/s]
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 8.6 MB/s eta 0:00:00


In [1]:
# include libraries
#import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torchvision.io import read_image
from PIL import Image
import torchvision.transforms as T

# check if gpu available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


cuda:0


In [ ]:
# custom dataset - returns watermarked image as image, and watermark free as label.

class Watermark_Dataset(Dataset):

  def __getitem__(self, idx):
      # get watermarked and corresponding watermark free image
      img_path = r"CLWD/train/Watermarked_image/" + str(idx + 1) + ".jpg"
      watermarked = Image.open(img_path).convert('RGB')
      img_path = r"CLWD/train/Watermark_free_image/" + str(idx + 1) + ".jpg"
      watermark_free = Image.open(img_path).convert('RGB')

      # convert to tensor
      transform_one = T.ToTensor()

      img = transform_one(watermarked)
      label = transform_one(watermark_free)

      # return img and label
      return img.float(), label.float()

  def __len__(self):
    #hardcoded length - number of watermarked/watermark free image pairs in datset
    return 60_000

# initialize dataset
Train_dataset = Watermark_Dataset()

In [ ]:
# Model class - convolutional autoencoder
# scales down image with encoding which is then decoded with decoder.
# takes in 256x256 JPG images and returns 256x256 JPG images (has 3 channels for RGB)
# at its lowest image is compressed to 29x29 image

class Auto_Encoder(nn.Module):
  def __init__(self):
    super().__init__()
    # encoder - compresses the image
    self.encoder = nn.Sequential(
        nn.Conv2d(3, 16, 3, stride = 2, padding = 1),
        nn.LeakyReLU(0.1),
        nn.Conv2d(16, 32, 3, stride = 2, padding = 1),
        nn.LeakyReLU(0.1),
        nn.Conv2d(32, 64, 7),
    )

    # max pooling - takes max of the pixels in an area to compress
    self.pool = nn.MaxPool2d(2, stride=2, return_indices=True)
    self.unpool = nn.MaxUnpool2d(2, stride=2)

    # decoder - decompresses
    self.decoder = nn.Sequential(
        nn.ConvTranspose2d(64, 32, 7),
        nn.LeakyReLU(0.1),
        nn.ConvTranspose2d(32, 16, 3, stride = 2, padding = 1, output_padding = 1),
        nn.LeakyReLU(0.1),
        nn.ConvTranspose2d(16, 3, 3, stride = 2, padding = 1, output_padding = 1),
        nn.Sigmoid()
    )
  # output
  def forward(self, x):

    encoded = self.encoder(x)

    # pooling and unpooling
    output, indices = self.pool(encoded)
    unpooled = self.unpool(output, indices)

    decoded = self.decoder(unpooled)
    return decoded


In [ ]:
# initializing the model, optimizer and loss type.
# move model and criterion to GPU if available
model = Auto_Encoder()
criterion = nn.MSELoss()
model.to(device)
criterion.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-2, weight_decay = 1e-4)


In [ ]:
# losses over the training epochs
losses = []

# initialize dataloader
train_dataloader = DataLoader(Train_dataset, batch_size=4, shuffle=True)
# * if GPU is good enough num_workers argument can be added and set to ~10, depending on resources

# see GPU stats just before training
!nvidia-smi
print(device)


# Training loop
transform = T.ToPILImage()
# if name == main makes windows system run GPU when sometimes it otherwise will not see the GPU, i dont really know why
# but thats what the forums said and it works
if __name__ == '__main__':
  epochs = 12
  for e in range(epochs):
      counter = 0
      running_loss = 0
      for images, labels in train_dataloader:
          # print progress
          counter += 1
          if counter % (37.5*4) == 0:
            print("training at epoch ", e, " ", (counter / (3750 * 4)) * 100, "% done")

          # data to GPU
          images = images.to(device)
          labels = labels.to(device)

          # get output from autoencoder
          output = model(images)

          # Data to GPU
          output.to(device)

          # backpropagation and caculate loss
          optimizer.zero_grad()
          loss = criterion(output, labels)
          loss.backward()
          optimizer.step()

          # keep track of loss
          running_loss += loss.item()

      losses.append((running_loss)/15000)

      # after each epoch, save model state to file to be tested later on
      torch.save({
              'epoch': e,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              'loss': loss,
              }, f"seven-ice_more_epoch-{e}.pth")

# print losses over the epochs
print(losses)

Thu Jun 22 21:46:24 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.18                 Driver Version: 531.18       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1660 S...  WDDM | 00000000:2B:00.0  On |                  N/A |
| 43%   41C    P2               43W / 125W|   1411MiB /  6144MiB |      6%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

KeyboardInterrupt: ignored